In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import io
import unicodedata
import re
from sklearn.model_selection import train_test_split

In [2]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [3]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [5]:
en, sp = create_dataset("spa-eng/spa-eng/spa.txt", 40000)
print(en[-1])
print(sp[-1])

<start> i chose the wrong answer . <end>
<start> escogi la respuesta incorrecta . <end>


In [6]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [7]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [8]:
# Try experimenting with the size of that dataset
num_examples = 40000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset("spa-eng/spa-eng/spa.txt", num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [20]:
max_length_inp , max_length_inp

(16, 16)

In [22]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

32000 32000 8000 8000


In [24]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [25]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 16]), TensorShape([64, 12]))

AttributeError: 'Encoder' object has no attribute 'batch_sz'

In [10]:
from tensorflow.keras.layers import *

In [90]:
class Encoder(keras.Model):
    def __init__(self , input_vocab , embedding_size , units, batch_size):
        super(Encoder , self).__init__()
        self.input_vocab = input_vocab
        self.embedding_size = embedding_size
        self.units = units
        self.batch_size = batch_size
        self.embedding = Embedding(input_vocab , embedding_size)
        self.rnn = GRU(units , 
                       return_sequences = True,
                       return_state = True
                       )
        
    def call(self , x , hidden):
        embedding = self.embedding(x)
        #embedding_size  = (batch_size , max_len , embedding_size)
        
        output , state = self.rnn(embedding)
        #output_shape = (batch_size , max_len , units)
        #state_shape = (batch_size , units)
        return output , state
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.units))

In [91]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 16, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [92]:
class BahdanauAttention(Layer):
    def __init__(self , units):
        super(BahdanauAttention , self).__init__()
        self.units = units
        self.W1 = Dense(self.units)
        self.W2 = Dense(self.units)
        self.V = Dense(1)
        
    def call(self , query , values):
        #query shape : (batch_size , hidden)
        #values shape : (batch_size , max_len , hidden)
        #query with timestamp : (batch_size , 1 , hidden)
        
        query = tf.expand_dims(query , 1)
        #query_shape ==> (batch_size , 1 , hidden)
        
        score = self.V(tf.nn.tanh(self.W1(query) + self.W2(values)))
        #self.W1 shape ==> (batch_size , 1 , units)
        #self.W2 shape ==> (batch_size , max_len , units)
        #Before applying self.V ==> score shape ==> (batch_size ,max_len , units)
        #After applying self.V ==> score shape ==> (batch_size , max_len , 1)
        
        attention_score = tf.nn.softmax(score , axis = 1)
        #attention shape ==> (batch_size , max_len , 1)
        
        #contex vector shape before sum ==> (batch_size , max_len , hidden)
        contex_vector = attention_score * values
        contex_vector = tf.reduce_sum(contex_vector , axis = 1)
        #contex vector shape after sum ==> (batch_size , hidden)
        
        return contex_vector , attention_score
        
        

In [93]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Contex vector shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Contex vector shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 16, 1)


In [94]:
class Decoder(keras.Model):
    def __init__(self , vocab_size , embedding_size , units , batch_size):
        super(Decoder , self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.units = units
        self.embedding = Embedding(vocab_size , embedding_size)
        self.rnn = GRU(units ,
                      return_sequences = True,
                      return_state = True)
        self.attention = BahdanauAttention(units)
        self.fc = Dense(vocab_size)
        
        
    def call(self , x , hidden , enc_output):
        #encoder output shape ==> (batch_size , max_len , hidden_size)
        # basically ==> [ embedding_size == hidden_size ] or for concatenate ==> it will be concataneted in along 3rd dim
        
        contex_vector , attention_weights = self.attention(hidden , enc_output)
        #contex vector ==> (batch_size , hidden)
        #attention weights ==> (batch_size , max_len , hidden)
        
        # x ==> (batch_size , 1)
        embedding = self.embedding(x)
        #embedding shape ==> (batch_size , 1 , embedding_size)
        
        embedding = tf.concat([tf.expand_dims(contex_vector , 1) , embedding] , axis = -1)
        #embedding shape after concatenate ==> (batch_size , 1 , embedding_size + hidden)
        
        output , state = self.rnn(embedding)
        
        #reshaping output for prediction
        output = tf.reshape(output , shape = (-1 , output.shape[2]))
        #output shape ==> (batch_size , embedding+hidden)
        
        final = self.fc(output)
        #final shape ==> (batch_size , target_vocab)
        
        return final , state , attention_weights

In [95]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 5934)


In [96]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [97]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [98]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [99]:
import time
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.6499
Epoch 1 Batch 100 Loss 2.2241
Epoch 1 Batch 200 Loss 1.9129
Epoch 1 Batch 300 Loss 1.8259
Epoch 1 Batch 400 Loss 1.5611
Epoch 1 Loss 1.9305
Time taken for 1 epoch 832.0164604187012 sec

Epoch 2 Batch 0 Loss 1.4594


KeyboardInterrupt: 